# K-mean clustering
How does k-mean clustering work?    
Take a look at the animations from the links below.    
http://tech.nitoyon.com/en/blog/2013/11/07/k-means/    
https://www.learndatasci.com/tutorials/k-means-clustering-algorithms-python-intro/

In [ ]:
# imports for this notebook
%matplotlib inline
#%matplotlib notebook
from copy import deepcopy
import numpy as np
import pandas as pd
import geopandas as gpd
import time
import mplleaflet
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')
colors = ['r', 'g', 'b', 'y', 'c', 'm', 'r', 'g', 'b']

# Euclidean Distance Caculator
def dist(a, b, ax=1):
    return np.linalg.norm(a - b, axis=ax)

We will create an interactive example of kmean clustering.    
First we create some random sample data around three centers and plot them.    
You can execute this multiple times to generate different datasets. Continue if your confident with it.

In [ ]:
# Set centers
n_center = 3
center= np.random.randn(n_center, 2)

# Generate random data and center it to the three centers
data = np.empty([1,2])
for i in range(n_center):
    data = np.append(data, np.random.randn(200, 2) + center[i,:], axis=0)

data = pd.DataFrame(data=data[1:],
            index=[i for i in range(data.shape[0]-1)],
            columns=['x', 'y'])

# Getting the values and plotting it
f1 = data.x.values
f2 = data.y.values
X = np.array(list(zip(f1, f2)))
fig, axes = plt.subplots()
axes.scatter(f1, f2, c='black', s=7)
axes.scatter(center[:,0],center[:,1], marker='o', s=200, c='r')
fig.set_size_inches(8,8)

Now we will try to find the centers of the random data by k-mean clustering.    
We create three random points as start center of the clusters and will move them until the error is minimal

In [ ]:
# Number of clusters
k = 3
# X coordinates of random centroids
C_x = np.random.randint(np.min(X), np.max(X), size=k)
# Y coordinates of random centroids
C_y = np.random.randint(np.min(X), np.max(X), size=k)
C = np.array(list(zip(C_x, C_y)), dtype=np.float32)
# Plotting along with the Centroids
fig, axes = plt.subplots()
axes.scatter(f1, f2, c='#050505', s=7)
axes.scatter(C_x, C_y, marker='o', s=200, c='g')
fig.set_size_inches(8,8)

# To store the value of centroids when it updates
C_old = np.zeros(C.shape)
# Cluster Lables(0, 1, 2)
clusters = np.zeros(len(X))
# Error func. - Distance between new centroids and old centroids
error = dist(C, C_old, None)

This a kind of animated (replots every second). Execute the cell and watch the cluster centers mooving.

In [ ]:
# Loop will run til the error becomes zero = centroids stop moving
fig, ax = plt.subplots()
fig.show()
fig.canvas.draw()
print("Error:")
#while error != 0: # automatic
if True: # manual steps
    #ax.clear()
    # Assigning each value to its closest cluster
    for i in range(len(X)):
        distances = dist(X[i], C)
        cluster = np.argmin(distances)
        clusters[i] = cluster
    # Storing the old centroid values
    C_old = deepcopy(C)
    # Finding the new centroids by taking the average value
    for i in range(k):
        points = [X[j] for j in range(len(X)) if clusters[j] == i]
        C[i] = np.mean(points, axis=0)
    
    for i in range(k):
        points = np.array([X[j] for j in range(len(X)) if clusters[j] == i])
        ax.scatter(points[:, 0], points[:, 1], s=7, c=colors[i])
        ax.scatter(C[:, 0], C[:, 1], marker='o', s=200, c='m')
        ax.scatter(center[:,0],center[:,1], marker='o', s=200, c='r')
        error = dist(C, C_old, None)
    print(error)
    fig.set_size_inches(10,10)
    fig.canvas.draw()
    #time.sleep(1)

# DBSCAN Cluserting
Have a look on the animation about how dbscan works    
https://www.naftaliharris.com/blog/visualizing-dbscan-clustering/

## Road Accidents
the geodataframe in accidents.p contains data from https://unfallatlas.statistikportal.de/_opendata.html     
The data is already fimported and saved in the pickle file.
Filter the data by acciedents where cars are involved (IstPKW) and try to find clusters.
Try different settings for epsilon and the minimal number of points for a cluster. What generates the best results?

In [ ]:
import contextily as ctx
from sklearn.cluster import DBSCAN
from sklearn import metrics
#from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler

df_accidents = pd.read_pickle('data/accidents.p')
display(df_accidents.head())

In [ ]:
# Funtion to add a contextily base map in the background

#our coordinates are in epsg 4326
# for correct basemap creation, they need to be in epsg 3857

ST_TONER = 'http://tile.stamen.com/toner/{z}/{x}/{y}.png'
ST_TONER_HYBRID = 'http://tile.stamen.com/toner-hybrid/{z}/{x}/{y}.png'
ST_TONER_LABELS = 'http://tile.stamen.com/toner-labels/{z}/{x}/{y}.png'
ST_TONER_LINES = 'http://tile.stamen.com/toner-lines/{z}/{x}/{y}.png'
ST_TONER_BACKGROUND = 'http://tile.stamen.com/toner-background/{z}/{x}/{y}.png'
ST_TONER_LITE = 'http://tile.stamen.com/toner-lite/{z}/{x}/{y}.png'

def add_basemap(ax, zoom, source='http://tile.stamen.com/terrain/{z}/{x}/{y}.png'):
    xmin, xmax, ymin, ymax = ax.axis()
    basemap, extent = ctx.bounds2img(xmin, ymin, xmax, ymax, zoom=zoom, source=source)
    ax.imshow(basemap, extent=extent, interpolation='bilinear')
    # restore original x/y limits
    ax.axis((xmin, xmax, ymin, ymax))

In [ ]:
df_accidents['x'] = df_accidents.geometry.x
df_accidents['y'] = df_accidents.geometry.y

condition = df_accidents.IstPKW=='1'
fdata = df_accidents[condition]
X= fdata[['x', 'y']].to_numpy()
#fdata.plot()

# #############################################################################
# Compute DBSCAN
db = DBSCAN(eps=0.0010, min_samples=8).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

# Black removed and is used for noise instead.
fig, ax = plt.subplots()
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    ax.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),markersize=5)

    xy = X[class_member_mask & ~core_samples_mask]
    ax.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col), markersize=1)

ax.set_title('Estimated number of clusters: %d' % n_clusters_)
add_basemap(ax,15)
#ctx.add_basemap(ax, crs = 4326); # not working anymore?
ax.set_axis_off()
fig.show()
#mplleaflet.display(fig=fig, crs=df_accidents.crs) # alternative way to show points on map ?


## Try another clustering method: agglomerative clustering
Implement agglomerative clustering for the same dataset. Try different settings for the linkage method.    
See the referance for datails:
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn.cluster.AgglomerativeClustering     
Try different parameters to find a good setting.

In [ ]:
from sklearn.cluster import AgglomerativeClustering
treshold = 10 # number of points in cluster to be a relevant location
cluster = AgglomerativeClustering(n_clusters = None, distance_threshold = 0.025, compute_full_tree= True, linkage = 'complete').fit(X)
core_samples_mask = np.zeros_like(cluster.labels_, dtype=bool)
#core_samples_mask[cluster.core_sample_indices_] = True
labels = cluster.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

# Black removed and is used for noise instead.
fig, ax = plt.subplots(figsize = (20,10))
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)
   # print(tuple(col))
    #print()
    if X[class_member_mask].size > treshold:

        # middle of cluster
        xy = X[class_member_mask].mean(axis=0)
        ax.plot(xy[0], xy[1], 'o', markerfacecolor=tuple(col), markeredgecolor= tuple(col),markersize=15)

        xy = X[class_member_mask & core_samples_mask]
        ax.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col), markeredgecolor= tuple(col),markersize=5)

        xy = X[class_member_mask & ~core_samples_mask]
        ax.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col), markeredgecolor= tuple(col), markersize=3)

ax.set_title('Estimated number of clusters: %d' % n_clusters_)
add_basemap(ax,12)